This notebook is to just explore the baseline with an agent that works on a simple research based logic and to get acquainted with the environment.

**Please upvote it if you find it helpful or interesting!**

In [ ]:
%matplotlib inline

I would like to try out some researched logic in order to play the game. The link to the paper is [here](https://arxiv.org/pdf/1404.5199v1.pdf) and article is [here](https://arstechnica.com/science/2014/05/win-at-rock-paper-scissors-by-knowing-thy-opponent/#:~:text=Therefore%2C%20this%20is%20the%20best,thing%20that%20you%20just%20played.).

As per the research story goes, a group of researchers from Chinese universities have written a paper about the role of psychology in winning (or losing) at rock-paper-scissors. After studying how players change or keep their strategies during multiple-round sessions, they figured out a basic rule that people tend to play by that could potentially be exploited.

The researchers took 360 students, broke them into groups of six, and had them play 300 rounds of rock-paper-scissors in random pairings. The students received small amounts of money each time they won a round. As they played, the researchers observed how the players rotated through the three play options as they won or lost. It was done to extract and formulate a simple pattern of behavioural psychology which mostly works for individuals playing the game.

### Logic to play with

Conclusively, the general pattern that was observed here and can be used as a winning strategy is: if you lose the first round, switch to the thing that beats the thing your opponent just played. If you win, don't keep playing the same thing, but instead switch to the thing that would beat the thing that you just played. In other words, play the hand your losing opponent just played. To wit: you win a round with rock against someone else's scissors. They are about to switch to paper. You should switch to scissors. Got it? Good.

This makes sense as, our human brain is 'mindlessly' into searching for patterns in anything. [Humans try to detect patterns in their environment all the time](https://www.sciencedaily.com/releases/2018/05/180531114642.htm). Now the name of the renowned book of Christopher Bishop makes more sense, doesn't it :P? Anyway, so here logic is to take advantage of this pattern recognition attribute and this is what we are going to do here.

The logic is going to do just fine as long the opponent doesn't know about it. So, kaggle bots don't you dare look here. :P

In [ ]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [ ]:
%%write_and_run submission.py

import numpy as np
import pandas as pd
import random

from kaggle_environments.envs.rps.utils import get_score

my_prev_move = None

def dont_always_copy_opponent_move(observation, configuration):
    """
    Implemented research logic.
    
    Info:
    Rock = 0, Paper = 1, Scissors = 2...
    """
    global my_prev_move
    N = configuration.signs
    curr_move = random.randrange(0, N)
    
    if observation.step == 0:
        my_prev_move = curr_move
        return curr_move
    else:
        opponent_prev_move = observation.lastOpponentAction
        
        result = get_score(my_prev_move, opponent_prev_move)
        
        if result == -1:    ##If we lose in the previous round
            curr_move = (opponent_prev_move + 1) % N
        elif result == 1:      ##If we win in previous round
            curr_move = opponent_prev_move
        else:               ##If there's a draw, we choose to go with random for now
            curr_move = random.randrange(0, N)
        
    my_prev_move = curr_move
    
    return curr_move

In [ ]:
from kaggle_environments import make

# Setup a rps environment.
env = make("rps", configuration={"episodeSteps": 1000})

# Run the basic agent against a default agent which chooses a "random" move.
env.run([dont_always_copy_opponent_move, "reactionary"])

# Render an html ipython replay of the tictactoe game.
env.render(mode="ipython", width=800, height=800)

Let me know if something can be corrected or improved. Thank you!